## Code Mess for Nanogenmo Google Maps/Places Stories

By Lynn Cherny, will be refactored as usual

In [1]:
import random
import datetime
import sys
from functools import wraps
from openai import OpenAI
import googlemaps
import vertexai
from vertexai.language_models import TextGenerationModel

API_KEY = 'your key'  # this is the Google key
OPENAI_API_KEY = 'your key'

In [4]:
# Assuming a picture is worth a 1000 words.  Treating words as just space separated even in markdown.

def wordcount(filename):
    with open(filename) as handle:
        text = handle.read()
    strings = text.split(" ")
    print(len(strings))
    pics = [x for x in strings if 'pics' in x]
    print(len(pics))
    print("total", 1000*len(pics) + len(strings))

wordcount("Barcelona1.md")

13242
43
total 56242


In [1172]:
# run this, then eval the code below to set vars right before running the "runner" line

!rm "Barcelona1.md"
MD_FILENAME = "Barcelona1.md"
!mkdir pics/barca
PICS_FOLDER = './pics/barca'
CODE = "barca"
LOCSTRING = "La Rambla, Barcelona, Spain"
CITYSTRING = "Barcelona, Spain" 
AI = True
DAY = 1
#REASON = get_random(reasons, n=1)  # or set to what you want, which is more fun.
REASON = reasons[3]
places_visited = {}


rm: Barcelona1.md: No such file or directory


In [1174]:
##  This can't be run until you re-eval the code underneath. Needs a refactor into actual py files.

vertexai.init(project="imageapi-172912", location="us-central1")
client = OpenAI(api_key=OPENAI_API_KEY)
model = TextGenerationModel.from_pretrained("text-bison")
gmaps = googlemaps.Client(key=API_KEY)

runner(LOCSTRING, morale=10, money=45)

In [1173]:
# re-eval this after running the cell with defs above!

def print_to_file(filename):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Save the current standard output
            original_stdout = sys.stdout
            if filename:
                # Redirect standard output to the specified file
                with open(filename, 'a') as f:
                    sys.stdout = f
                    try:
                        # Execute the function
                        result = func(*args, **kwargs)
                    finally:
                        # Restore the original standard output
                        sys.stdout = original_stdout
                # Return the function's result
                return result
        return wrapper
    return decorator


def get_pid_loc(geocode_res):
    pid = geocode_res['place_id']
    lat_lon = geocode_res['geometry']['location']
    return pid, lat_lon

#####  AI code - if it's "ai" and not "openAI" it's calling Google PALM.

def ai_intro_place(locstring=CITYSTRING, reason="You're trying to stop thinking about an ex."):
    parameters = {
    "candidate_count": 1,
    "max_output_tokens": 2000,
    "temperature": 0.8,
    "top_p": 0.9,
    "top_k": 40,
    "stop_sequences": ["\n"]
    }
    prompt = f"""You've come on vacation alone to {locstring}. {reason} Write a short paragraph explaining this in second person, 
        suggesting some things you are considering doing and seeing here that will help with this goal, as if you were talking to yourself.
        End with a newline."""
    response = model.predict(prompt,
        **parameters
    )
    text = response.text.strip()
    return text


def ai_rewrite_journal(text, locstring=CITYSTRING, reason=REASON):
    parameters = {
    "candidate_count": 3,
    "max_output_tokens": 2000,
    "temperature": 1,
    "top_p": 0.9,
    "top_k": 40,
    "stop_sequences": ["\n"]
    }
    prompt = f"""You are on vacation alone in {locstring}. You have just visited a destination here, and you're thinking about why you are here. {reason}
        Turn the destination's site review into an informal diary entry about what happened to you when you went there, turning the review facts into events that happened to you.
        Report the interesting events and observations in the review as if they happened to you, and if necessary make them more interesting!
        You are writing a short story paragraph, not a review with advice or prices. Do not mention family members or children. Do not repeat that you are on vacation alone in {locstring}. Don't mention if you recommend it or not. 
        End with a newline.
        Review: {text}
        Story:"""
    response = model.predict(prompt,
        **parameters
    )
    text = response.text.strip()
    if not text or len(text) == 0:
        return "Nothing worth writing about happened here."
    else:
        return text


def ai_rewrite_social(text, locstring=CITYSTRING):
    parameters = {
    "candidate_count": 1,
    "max_output_tokens": 1000,
    "temperature": 0.9,
    "top_p": 0.9,
    "top_k": 40,
    "stop_sequences": ["\n"]
    }
    prompt = f"""You are posting about your day on social media, but you want to give it a happy positive spin, even if it was bad!
    Turn your journal entry into a short, fun post on Instagram, with emoji if you want. Write only in English.
        End with a newline.
        Journal Entry: {text}
        Post:"""
    response = model.predict(prompt,
        **parameters
    )
    text = response.text.strip()
    if not text or len(text) == 0:
        return "Nothing worth posting about today, so busy having a great time!"
    else:
        return text


def openai_filler_morale_weather(moralestring, weather, time="morning", locstring=CITYSTRING, reason=REASON):
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {
          "role": "system",
          "content": f"""BACKGROUND FACTS: You are spending the next few hours in {locstring}. The time is {time}. Your mood is {moralestring}. The weather is {weather}. {reason}
TASK:
Write a few colorful sentences suggesting how you feel and what you might want to do given the weather now, in the second person. Do not repeat the location and reason for being here, but use them as direction.
Include details of the city activity around you, for atmosphere, and thoughts about activities related to your reason for being here. What do you see, hear, feel? What could you do now?
"""
        },
        {
          "role": "assistant",
          "content": "Create a short narrative paragraph in the second person with the background above."
        },
    
      ],
      temperature=1.12,
      max_tokens=500,
      top_p=1,
      frequency_penalty=0.11,
      presence_penalty=0
        )
    return response.choices[0].message.content


def ai_filler_morale_weather(moralestring, weather, time="morning", locstring=CITYSTRING, reason=REASON):
    parameters = {
    "candidate_count": 2,
    "max_output_tokens": 1024,
    "temperature": 0.9,
    "top_p": 0.8,
    "top_k": 40
    }
    prompt = f"""BACKGROUND FACTS: 
    You are spending the next few hours in {locstring}. It is {time}. Your mood is {moralestring}. The weather is {weather}. {reason}
    TASK:
    DO NOT repeat the location and reason for being here, but use them as direction. Write a few colorful sentences suggesting how you feel and what you might want to do given the weather now, in the second person.
    Include details of the city activity around you, for atmosphere, and thoughts about activities related to your reason for being here. What do you see, hear, feel? What could you do now?
    End with a newline.
    """
    response = model.predict(prompt,
        **parameters
    )
    return response.text.strip()


# not using, low quality from palm2
def ai_call_ending(morale, reason=REASON):
    parameters = {
    "candidate_count": 2,
    "max_output_tokens": 2020,
    "temperature": 1,
    "top_p": 0.8,
    "top_k": 40,
    "stop_sequences": ["\n"]
    }
    placenames = ','.join(places_visited.values())
    placenames = placenames.replace("\n", " ")
    if morale < 5:
        moralestring="low"
    elif morale >=5 and morale <=7:
        moralestring="okay"
    elif morale > 7:
        moralestring="high"
    reason_past = reason.replace("You're here", "You were here")
    prompt = f"""You have been on vacation and visited {placenames}. Your mood is now {moralestring}. {reason_past}
    Summarize your trip in the second person with original wording, and report how you feel now and what you saw.
    What were some highlights and low points?  Did you achieve your goal? What were some details about the trip that you liked or disliked? Be specific and tell a nice story."
    """
    response = model.predict(prompt,
        **parameters
    )
    return response.text.strip()


def openai_call_ending(morale, reason=REASON):
    placenames = ','.join(places_visited.values())
    placenames = placenames.replace("\n", " ")
    if morale < 5:
        moralestring="low"
    elif morale >=5 and morale <=7:
        moralestring="okay"
    elif morale > 7:
        moralestring="high"
    reason_past = reason.replace("You're here", "You were here")
    response = client.chat.completions.create(
      model="gpt-4",
      messages=[
            {
              "role": "system",
              "content": f"""You have been on vacation and visited {placenames}. Your mood is now {moralestring}. {reason_past}
        Summarize your trip in the second person with original wording, and report how you feel now and what you saw.
        What were some highlights and low points?  Did you achieve your goal? What were some details about the trip that you liked or disliked? Be specific and tell a nice story."
        """
            },
            {
              "role": "user",
              "content": f"What happened on my trip?",
            },
            {
              "role": "assistant",
              "content": "Create a short trip summary as described. You don't need to cover every place visited but do need to include details about the reason you were there."
            },
        
          ],
        temperature=1.12,
        max_tokens=1023,
        top_p=1,
        frequency_penalty=0.11,
        presence_penalty=0
            )
    return response.choices[0].message.content


def openai_rewrite_journal(text, locstring=CITYSTRING, reason=REASON):

    # This code is for v1 of the openai package: pypi.org/project/openai
    response = client.chat.completions.create(
      model="gpt-4",
      messages=[
        {
          "role": "system",
          "content": f""""You are on vacation alone in {CITYSTRING}. You have just visited a destination here, and you're thinking about why you are here. {reason}
Turn the destination's site review into an informal diary entry about what happened to you when you went there, turning the review facts into events that happened to you.
Report the interesting events and observations in the review as if they happened to you, and if necessary make them more interesting!
You are writing a short story paragraph, not a review with advice or prices. Do not mention family members or children or people with you.
Do not repeat that you are on vacation alone in {CITYSTRING}. Don't mention if you recommend the place or not.
"""
        },
        {
          "role": "user",
          "content": f"Review: {text}",
        },
        {
          "role": "assistant",
          "content": "Turn the review into a journal entry as described."
        },
    
      ],
      temperature=1.12,
      max_tokens=1023,
      top_p=1,
      frequency_penalty=0.11,
      presence_penalty=0
        )
    return response.choices[0].message.content

####################

def get_nearby(lat_lon, include=None, exclude=None, radius=5000):
    circle = {
             "center": {
                "latitude": lat_lon['lat'],
                "longitude": lat_lon['lng'],
             },
                "radius": radius
            }
    headers = {
        'X-Goog-Api-Key': API_KEY,
        "X-Goog-FieldMask": "places.id,places.rating,places.displayName,places.primaryType,places.userRatingCount,places.photos,places.location,places.priceLevel",
    }
    # {'lat': 51.50811239999999, 'lng': -0.0759493}
    res = requests.post("https://places.googleapis.com/v1/places:searchNearby",
                        headers = headers,
                          json =  {
                                      "includedTypes": include,
                                      "excludedPrimaryTypes": exclude,
                                      "maxResultCount": 20,
                                      "locationRestriction": {
                                            "circle": circle
                                        }
                                    }
                       )
    return res.json()

def download_photo(ref, filename):
    response = requests.get(f"https://places.googleapis.com/v1/{ref}/media?key={API_KEY}&maxWidthPx=500&maxHeightPx=500")
    if response.status_code == 200:
    # Open a file in binary write mode
        with open(filename, 'wb') as file:
            file.write(response.content)
    else:
        print("Failed to download image, status code:", response.status_code)


# doesn't really work right / not used
def get_nearby_city(lat_lon):
    url = f"https://maps.googleapis.com/maps/api/place/autocomplete/json?input=tourism"
    url += f"&location={lat_lon['lat'], lat_lon['lng']}&radius=20000&strictbounds=true&types=(cities)"
    url += f"&key={API_KEY}"
    #print(url)
    response = requests.get(url)
    return response.json()

def get_loc_from_place(place):
    return place['location']

def get_streetview(place, root_name, code=CODE, loc=None, prefix="You post a random snap of the sights to make your friends jealous:"):
    if not loc:
        lat_lon = get_loc_from_place(place)
    else:
        if 'latitude' not in loc:
            lat_lon = {'latitude': loc['lat'], 'longitude': loc['lng']}
    heading = get_random([90, 120, 0, 150, 180], n=1)
    fov = get_random([80, 100, 120], n=1)
    url = "https://maps.googleapis.com/maps/api/streetview?size=500x250"
    url += f"&location={lat_lon['latitude']:.3f},{lat_lon['longitude']:.3f}&heading={heading}&pitch=1&fov={fov}&key={API_KEY}"
    response = requests.get(url)
    if code:
        filename = f"{PICS_FOLDER}/{root_name}_{code}.jpg"
    else:
        filename = f"{PICS_FOLDER}/{root_name}.jpg"
    if response.status_code == 200:
    # Open a file in binary write mode
        with open(filename, 'wb') as file:
            file.write(response.content)
        return f"{prefix}\n![sv]({filename})\n"
    else:
        return None
    return

def timestamp():
    current_time = datetime.datetime.now()
    # Format the time. For example: '2023-11-28_13-45-30'
    timestamp = current_time.strftime("%H%M%S")
    return timestamp

def process_photo(photoinfo, saveid, code=CODE):
    name = photoinfo['name']
    author = photoinfo['authorAttributions'][0]['displayName']
    if code:
        root_name = f"{saveid}_{code}_{str(DAY)}"
    else:
        root_name = f"{saveid}_{str(DAY)}"
    download_photo(name, f"{PICS_FOLDER}/{root_name}.jpg")
    return author, f"{PICS_FOLDER}/{root_name}.jpg"

def format_image_link_md(text, link):
    if text and link:
        return f"\n![{text}]({link})\n\nImageAuthor: _{text}_ \n"
    else:
        return None

def get_random_place(places):
    global places_visited
    ids = {x['id']:x['displayName'] for x in places}
    remains = list(set(ids.keys()) - set(places_visited.keys()))
    id = get_random(remains, n=1)
    picked = [x for x in places if x['id'] == id][0]
    places_visited[picked['id']] = picked['displayName']['text']
    return picked
    

def get_random(inputlist, n=1, place=False):
    if place:
        return get_random_place(inputlist)
    if n==1:
        return random.sample(inputlist, n)[0]
    else:
        return random.sample(inputlist, n)


def extract_review_details(rev):
    return { 'text': rev.get('text', None),
             'rating': rev.get('rating', None)
           }
            
def get_reviews(pid, n=4):
    revs = gmaps.place(pid, 
                        fields=["editorial_summary", "review", "reviews", "name", "price_level", 
                                "rating", "user_ratings_total"])
    revs = revs['result']
    if 'editorial_summary' in revs:
        overall_rev = revs['editorial_summary'].get('overview', None)
        overall_rating = revs['editorial_summary'].get('rating', None)
        overall_name = revs['editorial_summary'].get('name', None)
    else:
        overall_rev = None
        overall_rating = None
        overall_name = None
    review_count = revs.get("user_ratings_total", 0)
    if review_count > 0:
        reviews = [extract_review_details(r) for r in revs['reviews']]
    else:
        reviews = []
    return {
        "overall": overall_rev,
        "rating": overall_rating,
        "name": overall_name,
        "rev_count": review_count,
        "revs": reviews[0:n]
    }
    

def get_hotel_by_money(loc, money):
    types = lodging_mid
    if money <=5 and money > 0:
        types = lodging_cheap
    elif money >= 9:
        types = lodging_high
    elif money <= 0:
        types = lodging_broke
    if types != lodging_high:
        exclude = ["resort_hotel", "private_guest_room"]
    else:
        exclude = []
    hits = get_nearby(loc, include=types, exclude=exclude)
    res = hits.get("places", None)
    if res:
        return get_random(res, n=1, place=True)
    else:
        #print("failed to get", type)
        return get_hotel_by_money(loc, 0)

def get_photo(place, fileid):
    pics = place.get('photos', None)
    if pics:
        photoref = get_random(pics, n=1)
        author, path = process_photo(photoref, fileid)
        return author, path
    else:
        return None, None


def clean_review(text):
    text = text.split("\n")
    text = [x for x in text if x.strip()]
    text = " ".join(text)
    return text.strip()


def random_review_string(rating):
    options = [
        f"You shrug and leave a {rating} review.",
        f"In any case, you leave them a {rating} rating.",
        f"You rate it a {rating}.",
        f"You consider this a {rating} star visit."]
    text = get_random(options, n=1)
    return "\n" + text + "\n"


def handle_reviews(reviews, morale, money, ai=AI):
    text = ""
    if reviews['overall']:
        text += f"\nThe guide says: \"*{reviews['overall'].strip()}*\"\n"
    yourluck = get_random(['good', 'bad'], n=1)
    if reviews['revs']:
        reviews = reviews['revs']
        if yourluck == "good":
            reviews.sort(key = lambda x: x['rating'], reverse=True)
        else:
            reviews.sort(key= lambda x: x['rating'], reverse=False)
        review = get_random(reviews[0:4], n=1)
        clean_text = clean_review(review['text'])
        if ai:
            text += f"\n<details><summary>Original Review Source</summary>\n{clean_text.strip()} \n</details>"
            revtext = openai_rewrite_journal(clean_text)  # use gpt4 for better text
        else:
            revtext = clean_text
        paras = '\t'.join(revtext.split("\n"))
        text += f"\nYou write about it in your journal:\n\n>{paras}\n"
        rating = review['rating']
        text += random_review_string(rating)
        posttext = ai_rewrite_social(clean_text) # fine for palm
        text += f"\n\nYour post on Facebook says:\n\n>*{posttext}*\n\n"
        if rating >= 4:
            morale += 1
            money -= 2
        else:
            morale -= 1
            money -= 1
    elif yourluck == "bad":
        morale -= 1
        money -= 2
    elif yourluck == "good":
        morale += 1
        money -= 1
    return morale, money, text

@print_to_file(MD_FILENAME)
def hotel_stuff(loc, morale, money):
    hotel = get_hotel_by_money(loc, money)
    print(f"## Day {DAY}\n\n")
    pid = hotel['id']
    if money <= 5:
        print(f"\nYou're running low on money. Changing it up a bit...")
    print(f"\nTonight you'll be sleeping here: _{hotel['displayName']['text']}_ \n")
    author, pic = get_photo(hotel, "hotel")
    print(format_image_link_md(author, pic))
    reviews = get_reviews(pid, n=20)
    morale, money, text = handle_reviews(reviews, morale, money)
    print(text)
    return morale, money

def get_weather_string(morale, ai=True, time="morning"):
    weather = get_random(["sunny", "raining", "cloudy", "snowing"], n=1)
    if weather == "sunny":
        morale += 1
    elif weather in ["raining", "snowing"]:
        morale =- 2
    elif weather == "cloudy":
        morale -= 1
    if morale <= 3:
        moralestring = "low"
    elif morale >3 and morale < 7:
        moralestring = "average"
    elif morale >=7:
        moralestring = "high"
    if ai:
        text = openai_filler_morale_weather(moralestring, weather, time=time) # better text
    else:
        text = f"It's {weather} and your energy is {moralestring}."
    return weather, morale, text
    
def past_time_by_weather(weather):
    types = hobbies + culture
    if weather in ["raining", "snowing"]:
        types = indoor_hobbies
    return types

@print_to_file(MD_FILENAME)
def past_time(loc, morale, money, exclude=None, time="morning"):
    always_exclude = ["hotel", "lodging", "restaurant"]
    if exclude:
        always_exclude.append(exclude)
    exclude = always_exclude
    weather, morale, weathertext = get_weather_string(morale, ai=AI, time=time)
    types = past_time_by_weather(weather)
    include = get_random(types, n=5)
    hits = get_nearby(loc, include=include, exclude=exclude)
    res = hits.get("places", None)
    if res:
        place = get_random(res, n=1, place=True)
    else:
        #print("failed to get types", types)
        return past_time(loc, morale, money, exclude=exclude)
    pid = place['id']
    typehit = place.get('primaryType', None)
    pictext = get_streetview(place, "past_time_"+pid[-5:])
    if pictext:
        print(pictext)
    print("\n***\n")
    print(weathertext)
    print(f"\nSo you decide to spend a few hours here: _{place['displayName']['text']}_ \n")
    filecode = "place_" + timestamp()
    author, pic = get_photo(place, filecode)
    print(format_image_link_md(author, pic))
    reviews = get_reviews(pid, n=20)
    morale, money, revtext = handle_reviews(reviews, morale, money)
    print(revtext)
    return morale, money, typehit

def change_loc(loc):
    types = ["tourist_attraction"]
    hits = get_nearby(loc, include=types, radius=20000)
    res = hits.get("places", None)
    if res:
        place = get_random(res, n=1, place=True)
        lat_lon = place['location']
        loc = {'lat': lat_lon['latitude'],
                'lng': lat_lon['longitude']
          }
    return loc
    

@print_to_file(MD_FILENAME)
def go_food(loc, morale, money):
    types = ["restaurant", "bar"]
    hits = get_nearby(loc, include=types, exclude=["hotel", "lodging"])
    res = hits.get("places", None)
    if res:
        place = get_random(res, n=1, place=True)
    else:
        #print("\n--Debug: failed to get types\n", types)
        return go_food(loc, morale, money)
    pid = place['id']
    print(f"\nYou get a bite here: _{place['displayName']['text']}_ \n")
    author, pic = get_photo(place, "resto")
    print(format_image_link_md(author, pic))
    reviews = get_reviews(pid, n=20)
    morale, money, revtext = handle_reviews(reviews, morale, money)
    print(revtext)
    return morale, money

@print_to_file(MD_FILENAME)
def spend_a_day(loc, morale, money):
    morale, money = hotel_stuff(loc, morale, money)
    #print(f"\n\t__Current morale: {morale}, money: {money}__")
    morale, money, typehit = past_time(loc, morale, money, time="morning")
    #print(f"\n\t__Current morale: {morale}, money: {money}__")
    morale, money = go_food(loc, morale, money)
    morale, money, _ = past_time(loc, morale, money, exclude=typehit, time="afternoon or evening")
    new_loc = change_loc(loc)
    #print(f"\n\t__Current morale: {morale}, money: {money}__")
    return morale, money, new_loc


@print_to_file(MD_FILENAME)
def print_ending(loc, morale, reason=REASON):
    print("\n### Heading Home!\n")
    text = openai_call_ending(morale, reason=reason)
    print(text)
    print("\nOf all the snaps you took, these were your favs:\n")
    pictext = get_streetview(None, "ending_"+timestamp(), loc=loc, prefix="")
    if pictext:
        print(pictext)
    res = gmaps.geocode(CITYSTRING)
    pid, loc = get_pid_loc(res[0])
    pictext = get_streetview(None, "ending_"+timestamp(), loc=loc, prefix="")
    if pictext:
        print(pictext)


@print_to_file(MD_FILENAME)
def print_location_intro(locationstring, reason=REASON):
    res = gmaps.geocode(locationstring)
    pid, loc = get_pid_loc(res[0])
    #print("---Debug, got location", loc, "\n")
    print(f"## Diary of a (Google Maps) Trip to {CITYSTRING}\n")
    intro_text = ai_intro_place(locstring=CITYSTRING, reason=reason)
    print(intro_text + "\n\n")
    return pid, loc

@print_to_file(MD_FILENAME)
def runner(locationstring, morale=10, money=10):
    global DAY
    pid, loc = print_location_intro(locationstring, reason=REASON)
    morale, money, loc = spend_a_day(loc, morale, money)
    while money > 0:
        DAY += 1
        morale, money, loc = spend_a_day(loc, morale, money)
    print_ending(loc, morale)


In [1171]:
### Lists for use in the code, from the google docs and my own reworks

sports = """athletic_field
fitness_center
golf_course
gym
playground
ski_resort
sports_club
sports_complex
stadium
swimming_pool""".split("\n")

outdoors = ["farm", "park", "dog_park", "hiking_area", "zoo", "playground", "national_park", "marina"]

shopping = """auto_parts_store
bicycle_store
book_store
cell_phone_store
clothing_store
convenience_store
department_store
discount_store
electronics_store
furniture_store
gift_shop
grocery_store
hardware_store
home_goods_store
home_improvement_store
jewelry_store
liquor_store
market
pet_store
shoe_store
shopping_mall
sporting_goods_store
store
supermarket
wholesaler""".split("\n")

hobbies = ["bowling_alley", "casino", "swimming_pool", "golf_course", "movie_theater", "amusement_park", "tourist_attraction", "historical_landmark"]

indoor_hobbies = ["bowling_alley", "casino", "movie_theater", "coffee_shop", "museum", "art_gallery", "bookstore", "gift_shop", "market", "shopping_mall", "fitness_center"]

nightime = ['bar', 'nightclub', 'casino', 'cocktail_bar']

culture = """art_gallery
museum
tourist_attraction
performing_arts_theater""".split("\n")

lodging = """bed_and_breakfast
campground
camping_cabin
cottage
extended_stay_hotel
farmstay
guest_house
hostel
hotel
lodging
motel
private_guest_room
resort_hotel
rv_park""".split("\n")


transport = """airport
bus_station
bus_stop
ferry_terminal
heliport
light_rail_station
park_and_ride
subway_station
taxi_stand
train_station
transit_depot
transit_station
truck_stop""".split("\n")

lodging_cheap = ['rv_park', 'motel', 'hostel', 'campground', 'camping_cabin']
lodging_mid = ['bed_and_breakfast', 'farmstay', 'guest_house', 'hotel', 'extended_stay_hotel']
lodging_high = ['resort_hotel', 'hotel', 'private_guest_room']
lodging_broke = ['bus_station', 'train_station', 'airport', 'truck_stop', 'park', 'subway_station']


reasons = ["You're here to try to forget about your ex.",
           "You're here because you are investigating a murder, and the deceased was here last.",
           "You're here because you are searching for a missing person.",
           "You're here to search for ghosts and poltergeists, because you are a paranormal investigator.",
           "You're here because your best friend bought you this trip when your hamster died, to try to cheer you up.", 
           "You're here to do research for a novel, to try to steep yourself in the history and soak up the atmosphere.",
           "You're here because you have a secret meeting with an occult circle.",
           "You're here because you have been working nonstop for months and it's time to relax.",
           "You're here to research an indie travel guide.",
           "You're here to meet an agent who is selling trade secrets, but you are under cover as a tourist."
          ]


# Google Docs Pages and Info


[Main Google Maps intro page](https://mapsplatform.google.com/maps-products/?_gl=1*oe1g7u*_ga*NzQ5ODcyODc0LjE3MDA4MzM1NTc.*_ga_NRWSTWS78N*MTcwMDgzMzU3NS4xLjEuMTcwMDgzNjE3NC4wLjAuMA..#places-section)


[Place Data Fields](https://developers.google.com/maps/documentation/places/web-service/place-data-fields)

[Place Types](https://developers.google.com/maps/documentation/places/web-service/supported_types)

[Place Photos (New API)](https://developers.google.com/maps/documentation/places/web-service/place-photos)

[StreetView Request](https://developers.google.com/maps/documentation/streetview/request-streetview)

[Python client for Maps](https://googlemaps.github.io/google-maps-services-python/docs/index.html) - note, i didn't think they all worked and sometimes resorted to raw requests with GET/POST, maybe out of date?





## fields for place

'serves_beer', 'business_status', 'geometry/location', 'geometry/viewport/northeast', 'type', 'name', 'user_ratings_total', 'geometry/location/lat', 'geometry/viewport/southwest', 'wheelchair_accessible_entrance', 'serves_dinner', 'geometry/viewport/northeast/lng', 'editorial_summary', 'price_level', 'formatted_address', 'serves_lunch', 'geometry/location/lng', 'geometry', 'photo', 'dine_in', 'icon', 'plus_code', 'international_phone_number', 'serves_vegetarian_food', 'opening_hours', 'secondary_opening_hours', 'serves_wine', 'geometry/viewport/southwest/lng', 'serves_brunch', 'geometry/viewport', 'curbside_pickup', 'permanently_closed', 'website', 'serves_breakfast', 'takeout', 'review', 'utc_offset', 'delivery', 'url', 'geometry/viewport/northeast/lat', 'address_component', 'current_opening_hours', 'place_id', 'formatted_phone_number', 'reviews', 'vicinity', 'geometry/viewport/southwest/lat', 'adr_address', 'reservable', 'rating', 


Types for a nearby search: https://developers.google.com/maps/documentation/places/web-service/place-types#table-a

## Places search

places(query=None, location=None, radius=None, language=None, min_price=None, max_price=None, open_now=False, type=None, region=None, page_token=None)
Places search.

Parameters:
query (string) – The text string on which to search, for example: “restaurant”.

location (string, dict, list, or tuple) – The latitude/longitude value for which you wish to obtain the closest, human-readable address.

radius (int) – Distance in meters within which to bias results.

language (string) – The language in which to return results.

min_price (int) – Restricts results to only those places with no less than this price level. Valid values are in the range from 0 (most affordable) to 4 (most expensive).

max_price (int) – Restricts results to only those places with no greater than this price level. Valid values are in the range from 0 (most affordable) to 4 (most expensive).

open_now (bool) – Return only those places that are open for business at the time the query is sent.

type (string) – Restricts the results to places matching the specified type. The full list of supported types is available here: https://developers.google.com/places/supported_types

region (string) – The region code, optional parameter. See more @ https://developers.google.com/places/web-service/search

page_token (string) – Token from a previous search that when provided will returns the next page of results for the same search.

Return type:
result dict with the following keys: results: list of places html_attributions: set of attributions which must be displayed next_page_token: token for retrieving the next page of results

In [1]:
reasons

NameError: name 'reasons' is not defined